# Learning PyCUDA

See my [notes about CUDA](https://rvoulon.github.io/DataScience/Learning-CUDA.html) on Github Pages

In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
a = np.random.randn(4,4)
a

array([[ 1.26036595, -1.24398514,  0.72293573, -0.29484343],
       [ 0.65007247, -0.96747296, -0.66077816, -1.96263409],
       [-0.6852697 ,  0.57902537, -0.65066865,  0.44548528],
       [-0.70910985,  0.1794732 ,  1.36934208,  0.80123096]])

NVIDIA devices only support single precision, so `float32`:

In [3]:
a = a.astype(np.float32)
a

array([[ 1.260366  , -1.2439852 ,  0.72293574, -0.29484344],
       [ 0.65007246, -0.967473  , -0.66077816, -1.9626341 ],
       [-0.6852697 ,  0.5790254 , -0.6506686 ,  0.44548526],
       [-0.70910984,  0.1794732 ,  1.3693421 ,  0.80123097]],
      dtype=float32)

Allocate memory on the device, and then copy our data (`a`) to it:

In [4]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [5]:
cuda.memcpy_htod(a_gpu, a)

Write our kernel: double each entry in `a_gpu`

In [6]:
mod = SourceModule("""
    __global__ void doublify(float *a) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        a[idx] =* 2;
    }
    """)

CompileError: nvcc compilation of /tmp/tmpykq9c0da/kernel.cu failed
[command: nvcc --cubin -arch sm_61 -I/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pycuda/cuda kernel.cu]
[stderr:
kernel.cu(5): error: operand of "*" must be a pointer

1 error detected in the compilation of "/tmp/tmpxft_00000e3a_00000000-6_kernel.cpp1.ii".
]

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85


In [8]:
!echo $PATH

/home/paperspace/anaconda3/envs/fastai/bin:/usr/local/cuda-9.1/bin:/home/paperspace/bin:/home/paperspace/.local/bin:/home/paperspace/anaconda3/envs/fastai/bin:/usr/local/cuda-9.1/bin:/home/paperspace/anaconda3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


## Ok so scratch that, let's try this:

In [9]:
mod = SourceModule("""
    __global__ void cuda_dbl(float *dest, float *a) {
        const int i = threadIdx.x;
        dest[i] = 2.*a[i];
    }
""")

In [10]:
cuda_dbl = mod.get_function("cuda_dbl")
a = np.ones(400, dtype=np.float32)
ans = np.zeros_like(a)

In [11]:
cuda_dbl(cuda.Out(ans), cuda.In(a), block=(400,1,1), grid=(1,1))

In [12]:
ans

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

### W00t!